#### Module import

In [44]:
import pandas as pd # 데이터 파일 불러오기 위한 패키지
import os # 폴더 경로 설정을 위한 패키지
import numpy as np # 데이터 배열 변환 및 계산을 위한 패키지
from sklearn.model_selection import train_test_split # 데이터 Train, Validation으로 나눠주기 위한 패키지
import statsmodels.api as sm # 로지스틱 회귀모형을 불러오기 위한 패키지
from firthlogist import FirthLogisticRegression

##### Data import

In [25]:
# train import
sas_train = pd.read_sas('C:/Users/Owner/Desktop/윤태준/소/3차 요청자료/230304_Analysis_도축개월28_35/testset.sas7bdat')
# test import
sas_val = pd.read_sas('C:/Users/Owner/Desktop/윤태준/소/3차 요청자료/230304_Analysis_도축개월28_35/validset.sas7bdat')

In [26]:
sas_train.GENDER = pd.Series(sas_train['GENDER']).astype('object')
sas_val.GENDER = pd.Series(sas_val['GENDER']).astype('object')

In [32]:
sas_train.head()

,ID,KPN,GENDER,BIR,FARM_ID,FARM_CLASS,SL_M,SL_D,CLASS,S_W,...,D_ID,D_S_M_W,D_S_M_I,D_S_F_M,D_S_T_M,D_S_C,FARM_LEVEL,TARGET1,TARGET2,Replicate
0,b'000189678601',b'KPN496',b'\xb0\xc5\xbc\xbc',b'2006-05-25',78238.0,b'1+B',32.0,b'2009-01-20',b'1++B',406.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
1,b'000189803362',b'KPN485',b'\xb0\xc5\xbc\xbc',b'2006-03-03',21396.0,b'1+C',35.0,b'2009-01-07',b'1B',546.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
2,b'000194536248',b'KPN496',b'\xbe\xcf',b'2006-06-01',99065.0,b'2B',32.0,b'2009-01-10',b'2B',274.0,...,NaN,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
3,b'000195425435',b'KPN363',b'\xb0\xc5\xbc\xbc',b'2006-04-19',73033.0,b'1+B',32.0,b'2008-12-03',b'1++B',457.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0
4,b'000196536482',b'KPN471',b'\xb0\xc5\xbc\xbc',b'2006-02-19',71971.0,b'1A',33.0,b'2008-10-24',b'1+C',490.0,...,NaN,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0


##### Modeling: Logistic Regression

In [56]:
# scaled Train set, Logistic Regression.fit(), C()로 category변수 지정, Treatment로 ref설정
model = sm.Logit.from_formula('''TARGET1 ~ C(GENDER) + C(FARM_LEVEL, Treatment(3)) + SL_M + S_W + S_M_W + S_F_M 
    + S_T_M + S_C + A_S_M_W + A_S_T_M + B_S_M_W 
    + B_S_M_I + B_S_T_M''', sas_train, C=1e8, penalty=None, solver='newton-cg', tol=0.00000001).fit()
print(model.summary())
print('\n')
# 오즈비 계산(Confidence interval 포함)
odds_ratios = pd.DataFrame(
    {
        "OR": model.params,
        "Lower CI": model.conf_int()[0],
        "Upper CI": model.conf_int()[1],
    }
)
odds_ratios = np.exp(odds_ratios)
print('Odds_ratio(Confidence Interval)')
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.594637
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                TARGET1   No. Observations:                52505
Model:                          Logit   Df Residuals:                    52490
Method:                           MLE   Df Model:                           14
Date:                Mon, 06 Mar 2023   Pseudo R-squ.:                 0.07380
Time:                        11:56:41   Log-Likelihood:                -31221.
converged:                       True   LL-Null:                       -33709.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -4.4252      0.217    

c:\Users\Owner\anaconda3\envs\'cow'\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['C', 'penalty', 'solver', 'tol']
  warnings.warn(msg, ValueWarning)
c:\Users\Owner\anaconda3\envs\'cow'\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['C', 'penalty', 'solver', 'tol']
  warnings.warn(msg, ValueWarning)


In [42]:
help(sm.Logit)

Help on class Logit in module statsmodels.discrete.discrete_model:

class Logit(BinaryModel)
 |  Logit(endog, exog, check_rank=True, **kwargs)
 |  
 |  Logit Model
 |  
 |  Parameters
 |  ----------
 |  endog : array_like
 |      A 1-d endogenous response variable. The dependent variable.
 |  exog : array_like
 |      A nobs x k array where `nobs` is the number of observations and `k`
 |      is the number of regressors. An intercept is not included by default
 |      and should be added by the user. See
 |      :func:`statsmodels.tools.add_constant`.
 |  missing : str
 |      Available options are 'none', 'drop', and 'raise'. If 'none', no nan
 |      checking is done. If 'drop', any observations with nans are dropped.
 |      If 'raise', an error is raised. Default is 'none'.
 |  check_rank : bool
 |      Check exog rank to determine model degrees of freedom. Default is
 |      True. Setting to False reduces model initialization time when
 |      exog.shape[1] is large.
 |  
 |  
 | 